In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split #pip install sklearn
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
import os
import cv2
from tensorflow import keras
tf.config.run_functions_eagerly(True)

In [ ]:
assert hasattr(tf, "function") # Be sure to use tensorflow 2.0

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class_names = ['plastique','papier','verre','metal','glace','déchet organique']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)
IMAGE_SIZE = (200,200)

print(class_names_label)
print(nb_classes)

{'UP': 0, 'LEFT': 1, 'RIGHT': 2}
3


In [ ]:
# loading the data

def load_data():
    DIRECTORY = r'/content/drive/MyDrive/classificationTest'
    CATHEGORY = ['seg_train']
    
    output = []
    
    for category in CATHEGORY:
        path = os.path.join(DIRECTORY, category)
        print(path)
        images = []
        labels = []
        
        print('loading {} '.format(category))
        
        for folder in os.listdir(path):
            label = class_names_label[folder]
            
            #iterate through each image in our folder
            for file in os.listdir(os.path.join(path, folder)):
                
                #Get the path name of the image 
                img_path = os.path.join(os.path.join(path, folder), file)
                
                #Open and resize the image
                image = cv2.imread(img_path)
                resized = cv2.resize(image, (250,250), cv2.INTER_AREA)
          # Correcting coloring for MatPlotLib
                image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
                
                #image = cv2.imread(img_path)
                #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #image = cv2.resize(image, IMAGE_SIZE)
                #print(img_path, '=>', label)
                #Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')
        
        #output.append((images, labels))
        
        
    return (images, labels)    

In [ ]:
images, targets = load_data() 

/content/drive/MyDrive/classificationTest/seg_train
loading seg_train 


In [ ]:
images, images_valid, targets, targets_valid = train_test_split(images, targets, test_size=0.20)
print("images.shape", images.shape)
print("targets.shape", targets.shape)

print("images_valid.shape", images_valid.shape)
print("targets_valid.shape", targets_valid.shape) 


images.shape (220, 250, 250)
targets.shape (220,)
images_valid.shape (56, 250, 250)
targets_valid.shape (56,)


In [ ]:
images_max = images.max()
valid_max = images_valid.max()

images_train = images/images_max 
image_valid = images_valid/valid_max  
 
print('after normalisation : Min={}, before max={}'.format(images.min(),images.max()))
print(images_train[0].shape)


after normalisation : Min=28.0, before max=255.0
(250, 250)


In [ ]:
print("Mean and std of images", images.mean(), images.std())
scaler = StandardScaler()
scaled_images = scaler.fit_transform(images.reshape(-1, 480*640))
scaled_images_valid = scaler.transform(images_valid.reshape(-1, 480*640))
#targets = scaler.fit_transform(targets.reshape(447,))
print("Mean and std of scaled images", scaled_images.mean(), scaled_images.std())

scaled_images = scaled_images.reshape(-1, 480, 640, 3)
scaled_images_valid = scaled_images_valid.reshape(-1, 480, 640, 3)
print(scaled_images.shape)
print(scaled_images_valid[0]) 


In [ ]:
images_train = images.astype('float')/255
images_valid = images_valid.astype('float')/255 


In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Input((480,640,3)) )

model.add(keras.layers.Conv2D(16, (3,3), activation="relu") )
model.add(keras.layers.MaxPooling2D((2,2)) )
model.add(keras.layers.Dropout(0.2)) 

model.add(keras.layers.Conv2D(32, (3,3), activation="relu") )
model.add(keras.layers.MaxPooling2D((2,2)) ) 
model.add(keras.layers.Dropout(0.2)) 

model.add(keras.layers.Conv2D(64, (3,3), activation="relu") ) 
model.add(keras.layers.MaxPooling2D((2,2)) )
model.add(keras.layers.Dropout(0.2)) 

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation="relu") ) 
model.add(keras.layers.Dropout(0.5)) 

model.add(keras.layers.Dense(200, activation="relu") )
model.add(keras.layers.Dropout(0.5)) 

model.add(keras.layers.Dense(3, activation="softmax"))

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(250,250,1)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    #
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
                                    tf.keras.layers.MaxPool2D(2,2),
                                    #
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(512, activation='relu'),
                                    ##
                                    tf.keras.layers.Dense(3, activation='softmax')
    
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

In [ ]:
epochs = 10
batch_size = 5 

history = model.fit(images_train, targets, 
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (images_valid, targets_valid))

Epoch 1/10
44/44 [==============================] - 4s 88ms/step - loss: 1.0702 - accuracy: 0.5045 - val_loss: 208.0218 - val_accuracy: 0.5000
Epoch 2/10
44/44 [==============================] - 3s 63ms/step - loss: 0.3927 - accuracy: 0.8773 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
44/44 [==============================] - 3s 62ms/step - loss: 0.0376 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
44/44 [==============================] - 3s 65ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
44/44 [==============================] - 3s 72ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
44/44 [==============================] - 3s 60ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
44/44 [==============================] - 3s 59ms/step - loss: 8.3258e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_

In [ ]:
pred = model.predict(images[0:5])
print(pred)
print(targets[0:5])

In [ ]:
model.save('/content/drive/MyDrive/sauvegarde/save11.h5')

In [ ]:
def load_data_test():
    DIRECTORY = r'/content/drive/MyDrive/classificationTest'
    CATHEGORY = ['seg_test']
    
    output = []
    
    for category in CATHEGORY:
        path = os.path.join(DIRECTORY, category)
        print(path)
        images = []
        labels = []
        
        print('loading {} '.format(category))
        
        for folder in os.listdir(path):
            label = class_names_label[folder]
            
            #iterate through each image in our folder
            for file in os.listdir(os.path.join(path, folder)):
                
                #Get the path name of the image 
                img_path = os.path.join(os.path.join(path, folder), file)
                
                #Open and resize the image
                image = cv2.imread(img_path)
                resized = cv2.resize(image, (640,480), cv2.INTER_AREA)
          # Correcting coloring for MatPlotLib
                image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
                
                #image = cv2.imread(img_path)
                #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #image = cv2.resize(image, IMAGE_SIZE)
                #print(img_path, '=>', label)
                #Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')
        
        #output.append((images, labels))
        
        
    return (images, labels)

In [ ]:
images_test, target_test = load_data_test()

/content/drive/MyDrive/classificationTest/seg_test
loading seg_test 


In [ ]:
print(images_test.shape)

(9, 480, 640)


In [ ]:
pred = model.predict(images_test[0:10])
print(pred)
print(target_test[0:10])

1/1 [==============================] - 0s 481ms/step
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[1 1 1 2 2 2 0 0 0]


In [ ]:
model.save('/content/drive/MyDrive/sauvegarde/save_02.h5')

In [ ]:

new_model = tf.keras.models.load_model('/content/drive/MyDrive/sauvegarde/save.h5')